In [ ]:
# Imports
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import DEFAULT_PLOTLY_COLORS as default_colors
import seaborn as sns
import matplotlib.pyplot as plt

from QSSBuilder import QSSBuilder


# Calculating steady states.

In [ ]:
qss = QSSBuilder.from_csv("MegAWES")


In [ ]:
qss.add_steady_states(
    {
        "vw_mps": np.unique(np.linspace(0, 30, 13)),
        "Lt_m": np.unique(np.linspace(500, 1500, 3, dtype=int)),
        "phi_deg": [0.0, 17.5],
        "beta_deg": [0.0, 30],
        "chi_deg": np.unique(np.linspace(0, 180, 5, dtype=int)),
        # TODO: Smarter way to find different Ftk_N that work than looping over all values.
        # Use dtype int for the merge into an existing DataFrame to be less error-prone.
        # Otherwise it sometimes couldn't match two floats with each other. Integer
        # precision is also more than enough here.
        # "Ftk_N": np.unique(np.logspace(1, 7, 103, dtype=int)),
        "Ftk_N": np.unique(np.arange(10, 1e7, 2500, dtype=int)),
    }
)
qss.save_df()


In [ ]:
qss_massless = QSSBuilder.from_csv("MegAWES_massless")


In [ ]:
qss_massless.add_steady_states(
    {
        "vw_mps": np.unique(np.linspace(0, 30, 13)),
        "Lt_m": np.unique(np.linspace(500, 1500, 3, dtype=int)),
        "phi_deg": [0.0, 17.5],
        "beta_deg": [0.0, 30],
        "chi_deg": np.unique(np.linspace(0, 180, 5, dtype=int)),
        # TODO: Smarter way to find different Ftk_N that work than looping over all values.
        # Use dtype int for the merge into an existing DataFrame to be less error-prone.
        # Otherwise it sometimes couldn't match two floats with each other. Integer
        # precision is also more than enough here.
        # "Ftk_N": np.unique(np.logspace(1, 7, 103, dtype=int)),
        "Ftk_N": np.unique(np.arange(10, 1e7, 2500, dtype=int)),
    }
)
qss_massless.save_df()


# Processing

In [ ]:
temp = qss.df.sort_values("P_W", ascending=False)
qss_star = temp.drop_duplicates(
    ["beta_deg", "phi_deg", "chi_deg", "Lt_m", "vw_mps"],
    keep="first",
    ignore_index=True,
).copy()
star_dict = {
    "Ftk_N": "Ftk_N_star",
    "f": "f_star",
    "Ftg_N": "Ftg_N_star",
    "P_W": "P_W_star",
    "vr_mps": "vr_mps_star",
}
qss_star = qss_star.rename(columns=star_dict)

temp = qss_massless.df.sort_values("P_W", ascending=False)
qss_massless_star = temp.drop_duplicates(
    ["beta_deg", "phi_deg", "chi_deg", "Lt_m", "vw_mps"],
    keep="first",
    ignore_index=True,
).copy()
qss_massless_star = qss_massless_star.rename(columns=star_dict)

qss_massless_analytical = pd.read_csv("../results/MegAWES_massless_analytical.csv")
qss_massless_analytical_star = pd.read_csv(
    "../results/MegAWES_massless_analytical_star.csv"
)

In [ ]:
qss_star["param_name"] = "MegAWES"
qss_massless_star["param_name"] = "MegAWES_massless"
qss_massless_analytical_star["param_name"] = "MegAWES_massless_analytical"
df_massless = pd.concat(
    [qss_massless_star, qss_massless_analytical_star], ignore_index=True
)
df = pd.concat(
    [qss_star, qss_massless_star, qss_massless_analytical_star], ignore_index=True
)


# Compare analytical and qsm massless

In [ ]:
import my_plotly_themes


In [ ]:
fig = px.line(
    df_massless,
    x="vw_mps",
    y="P_W_star",
    color="param_name",
    symbol="Lt_m",
    facet_col="beta_deg",
    facet_row="phi_deg",
)
# fig.write_image("../results/qsm_massless_vs_analytical.png")
fig.show()


# Plotting

In [ ]:
# Linewidth in latex is 15.75 cm, which is 15.75/2.54 inch. I make it a bit larger which
# makes the text a bit smaller, but the figure a little bigger.
FULLSIZE = (15.75 / 2.54 + 3, 4)
PARTSIZE = (0.7 * (15.75 / 2.54 + 3), 4)

# From awe_workshop.
# # Configure the default plotting settings
# size=13
# params = {'legend.fontsize': 'large',
#           'figure.figsize': (8,5),
#           'axes.labelsize': size,
#           'axes.titlesize': size,
#           'xtick.labelsize': size*0.85,
#           'ytick.labelsize': size*0.85,
#           'axes.titlepad': 25}
# plt.rcParams.update(params)

### Loyd

In [ ]:
# fFP graph for massless system for beta and phi equal to zero and a certain tether
# length.
temp = qss_massless_analytical_star[
    (qss_massless_analytical_star["beta_deg"] == 0)
    & (qss_massless_analytical_star["phi_deg"] == 0)
    & (qss_massless_analytical_star["Lt_m"] == 1000)
]

fig, axs = plt.subplots(1, 3, figsize=FULLSIZE)

for i, y in enumerate(["f_star", "Ftg_N_star", "P_W_star"]):
    sns.lineplot(temp, x="vw_mps", y=y, ax=axs[i])
    axs[i].grid()

plt.tight_layout()
plt.savefig("../results/fFP_Loyd.png")

In [ ]:
# vr-Ft Loyd
fig, ax = plt.subplots(1, 1, figsize=PARTSIZE)
sns.lineplot(temp, x="vr_mps_star", y="Ftg_N_star", ax=ax)
ax.grid()
plt.tight_layout()
plt.savefig("../results/vrFt_Loyd.png")

### Beta and phi

In [ ]:
# fFP graph for massless system for beta and phi equal to zero and a certain tether
# length.
temp = qss_massless_analytical_star[(qss_massless_analytical_star["Lt_m"] == 1000)]

fig, axs = plt.subplots(1, 3, figsize=FULLSIZE)

for i, y in enumerate(["f_star", "Ftg_N_star", "P_W_star"]):
    sns.lineplot(temp, x="vw_mps", y=y, hue="beta_deg", style="phi_deg", ax=axs[i])
    axs[i].grid()
    if i != 2:
        axs[i].get_legend().set_visible(False)

plt.tight_layout()
plt.savefig("../results/fFP_betaphi.png")


In [ ]:
# vr-Ft Loyd
fig, ax = plt.subplots(1, 1, figsize=PARTSIZE)
sns.lineplot(
    temp, x="vr_mps_star", y="Ftg_N_star", hue="beta_deg", style="phi_deg", ax=ax
)
ax.grid()
plt.tight_layout()
plt.savefig("../results/vrFt_betaphi.png")


## Tether length

In [ ]:
# fFP graph for massless system for beta and phi equal to zero and a certain tether
# length.
beta_deg_bar = 0
phi_deg_bar = 0
temp = qss_massless_analytical_star[
    (qss_massless_analytical_star["beta_deg"] == beta_deg_bar)
    & (qss_massless_analytical_star["phi_deg"] == phi_deg_bar)
]

fig, axs = plt.subplots(1, 3, figsize=FULLSIZE)
fig.suptitle(f"{beta_deg_bar=}, {phi_deg_bar=}")

for i, y in enumerate(["f_star", "Ftg_N_star", "P_W_star"]):
    sns.lineplot(temp, x="vw_mps", y=y, hue="Lt_m", ax=axs[i])
    axs[i].grid()
    if i != 2:
        axs[i].get_legend().set_visible(False)

plt.tight_layout()
plt.savefig("../results/fFP_Lt.png")

In [ ]:
# vr-Ft Loyd
fig, ax = plt.subplots(1, 1, figsize=PARTSIZE)
fig.suptitle(f"{beta_deg_bar=}, {phi_deg_bar=}")

sns.lineplot(temp, x="vr_mps_star", y="Ftg_N_star", hue="Lt_m", ax=ax)

ax.grid()
plt.tight_layout()
plt.savefig("../results/vrFt_Lt.png")


### Add mass, variable chi.

In [ ]:
# fFP graph
beta_deg_bar = 0
phi_deg_bar = 0
# chi_deg_bar = 90
Lt_m_bar = 1000
temp = qss_star[
    (qss_star["Lt_m"] == Lt_m_bar)
    & (qss_star["beta_deg"] == beta_deg_bar)
    & (qss_star["phi_deg"] == phi_deg_bar)
    # & (qss_star['chi_deg'] == chi_deg_bar)
]
temp2 = qss_massless_analytical_star[
    (qss_massless_analytical_star["Lt_m"] == Lt_m_bar)
    & (qss_massless_analytical_star["beta_deg"] == beta_deg_bar)
    & (qss_massless_analytical_star["phi_deg"] == phi_deg_bar)
]

fig, axs = plt.subplots(1, 3, figsize=FULLSIZE)
fig.suptitle(f"{beta_deg_bar=}, {phi_deg_bar=}, {Lt_m_bar=}")

for i, y in enumerate(["f_star", "Ftg_N_star", "P_W_star"]):
    sns.lineplot(temp2, x="vw_mps", y=y, ax=axs[i], label="massless")
    axs[i].lines[0].set_linestyle("--")
    sns.lineplot(temp, x="vw_mps", y=y, hue="chi_deg", ax=axs[i])
    axs[i].grid()
    if i != 2:
        axs[i].get_legend().set_visible(False)

plt.tight_layout()
plt.savefig("../results/fFP_mass.png")

In [ ]:
# vr-Ft
fig, ax = plt.subplots(1, 1, figsize=PARTSIZE)
fig.suptitle(f"{beta_deg_bar=}, {phi_deg_bar=}, {Lt_m_bar=}")

sns.lineplot(temp2, x="vr_mps_star", y="Ftg_N_star", ax=ax, label="massless")
ax.lines[0].set_linestyle("--")
sns.lineplot(temp, x="vr_mps_star", y="Ftg_N_star", hue="chi_deg", ax=ax)

ax.grid()
plt.tight_layout()
plt.savefig("../results/vrFt_mass.png")

### With mass, chi=90 but variable beta and phi again.
Massless underestimates the effect of beta and phi (right?)

In [ ]:
# fFP graph
beta_deg_bar = 30
phi_deg_bar = 17.5
chi_deg_bar = 90
Lt_m_bar = 1000
temp = qss_star[
    (qss_star["Lt_m"] == Lt_m_bar)
    & (qss_star["beta_deg"] == beta_deg_bar)
    & (qss_star["phi_deg"] == phi_deg_bar)
    & (qss_star["chi_deg"] == chi_deg_bar)
]
temp2 = qss_massless_analytical_star[
    (qss_massless_analytical_star["Lt_m"] == Lt_m_bar)
    & (qss_massless_analytical_star["beta_deg"] == beta_deg_bar)
    & (qss_massless_analytical_star["phi_deg"] == phi_deg_bar)
]

fig, axs = plt.subplots(1, 3, figsize=FULLSIZE)
fig.suptitle(f"{beta_deg_bar=}, {phi_deg_bar=}, {chi_deg_bar=}, {Lt_m_bar=}")

for i, y in enumerate(["f_star", "Ftg_N_star", "P_W_star"]):
    sns.lineplot(temp2, x="vw_mps", y=y, ax=axs[i], label="massless")
    axs[i].lines[0].set_linestyle("--")
    sns.lineplot(temp, x="vw_mps", y=y, ax=axs[i])
    axs[i].grid()
    if i != 2:
        axs[i].get_legend().set_visible(False)

plt.tight_layout()
plt.savefig("../results/fFP_mass_betaphi.png")

In [ ]:
# vr-Ft
fig, ax = plt.subplots(1, 1, figsize=PARTSIZE)
fig.suptitle(f"{beta_deg_bar=}, {phi_deg_bar=}, {Lt_m_bar=}")

sns.lineplot(temp2, x="vr_mps_star", y="Ftg_N_star", ax=ax, label="massless")
ax.lines[0].set_linestyle("--")
sns.lineplot(temp, x="vr_mps_star", y="Ftg_N_star", ax=ax, label="heavy")

ax.grid()
plt.tight_layout()
plt.savefig("../results/vrFt_mass_betaphi.png")

### Making the average operating condition

In [ ]:
# Fitting to the curve:
# theta_hat = (X.T X)^-1 X.T y
# temp2['ones'] = 1
# temp2['vr2_m2ps2_star'] = temp2['vr_mps_star']**2
# X = temp2[['ones', 'vr_mps_star', 'vr2_m2ps2_star']].values
Fmin = 0.25e6
Fmax = 1.0e6
fit_index = temp["Ftg_N_star"][
    (temp["Ftg_N_star"] > Fmin)
    & (temp["Ftg_N_star"] < Fmax)
    & (~temp["Ftg_N_star"].isna())
].index

vr = temp.loc[fit_index, "vr_mps_star"].values
vr2 = vr**2
ones = np.ones_like(vr)
X = np.asarray([ones, vr, vr2]).transpose()

y = temp.loc[fit_index, "Ftg_N_star"].values
theta_hat = np.dot(np.dot(np.linalg.inv(np.dot(X.transpose(), X)), X.transpose()), y)
theta_hat

In [ ]:
def abc(a, b, c):
    return (-b + np.sqrt(b**2 - 4 * a * c)) / (2 * a)

In [ ]:
# Need to find the vr for which we reach Flim.
vr_power_lim = abc(theta_hat[2], theta_hat[1], theta_hat[0] - Fmax)
print(vr_power_lim, Fmax, vr_power_lim * Fmax / 1e6)

In [ ]:
def optimal_winch(vr):
    F_star = theta_hat[0] + theta_hat[1] * vr + theta_hat[2] * vr**2

    F_star[vr < 0] = Fmin
    F_star = np.maximum(F_star, Fmin)
    # F_star = np.clip(F_star, Fmin, Fmax)
    return F_star

In [ ]:
# vr-Ft
fig, ax = plt.subplots(1, 1, figsize=PARTSIZE)
fig.suptitle(f"{beta_deg_bar=}, {phi_deg_bar=}, {chi_deg_bar=}, {Lt_m_bar=}")

sns.lineplot(temp2, x="vr_mps_star", y="Ftg_N_star", ax=ax, label="massless")
ax.lines[0].set_linestyle("--")
sns.lineplot(temp, x="vr_mps_star", y="Ftg_N_star", ax=ax, label="heavy")
vr = np.linspace(-6, 8, 100)
sns.lineplot(x=vr, y=optimal_winch(vr), label="winch control with pull-up")

plt.ylim([0, 2.5e6])
plt.xlim([-6, 8])


ax.grid()
plt.tight_layout()
plt.savefig("../results/vrFt_mass_betaphi_fit.png")


vr_pos = np.linspace(0.1, 8, 100)
sns.lineplot(
    x=vr_pos,
    y=Fmax * vr_power_lim / vr_pos,
    label=f"power limit of {Fmax*vr_power_lim/1e6:.2} MW",
)
plt.hlines(1e6, -6, 8, color="purple", linestyles="--", label="Ft_max")
plt.legend()
plt.savefig("../results/vrFt_mass_betaphi_fit_power.png")

In [ ]:
temp


# System identification

In [ ]:
beta_deg_bar = 30
phi_deg_bar = 17.5
chi_deg_bar = 90
Lt_m_bar = 1000
qss_bar = qss.df[
    (qss.df["Lt_m"] == Lt_m_bar)
    & (qss.df["beta_deg"] == beta_deg_bar)
    & (qss.df["phi_deg"] == phi_deg_bar)
    & (qss.df["chi_deg"] == chi_deg_bar)
]

In [ ]:
# TODO: move SI from above to here.
beta_deg_bar = 30
phi_deg_bar = 17.5
chi_deg_bar = 90
Lt_m_bar = 1000
temp = qss.df[
    (qss.df["Lt_m"] == Lt_m_bar)
    & (qss.df["beta_deg"] == beta_deg_bar)
    & (qss.df["phi_deg"] == phi_deg_bar)
    & (qss.df["chi_deg"] == chi_deg_bar)
]

temp_nona = temp.dropna()

# Building the X matrix. (input data)
vr = temp_nona["vr_mps"].values
vr2 = vr**2
vw = temp_nona["vw_mps"].values
vw2 = vw**2
ones = np.ones_like(vr)
X = np.asarray([ones, vr, vr2, vw, vw2]).transpose()

# Outputs.
y = temp_nona["Ftg_N"].values

# Least-squares.
theta_hat = np.dot(np.dot(np.linalg.inv(np.dot(X.transpose(), X)), X.transpose()), y)
theta_hat

In [ ]:
# Doing the same with seaborn is really difficult.
fig, ax = plt.subplots(1, 1, figsize=PARTSIZE)

df_optimal = qss_star[
    (qss_star["Lt_m"] == Lt_m_bar)
    & (qss_star["beta_deg"] == beta_deg_bar)
    & (qss_star["phi_deg"] == phi_deg_bar)
    & (qss_star["chi_deg"] == chi_deg_bar)
]

sns.lineplot(
    df_optimal,
    x="vr_mps_star",
    y="Ftg_N_star",
)
ax.legend(["Optimal"])

beta_deg_bar = 30
phi_deg_bar = 17.5
chi_deg_bar = 90
Lt_m_bar = 1000
temp = qss_bar[
    (qss_bar["vr_mps"] > -6)
    & (qss_bar["vr_mps"] < 8)
    & (qss_bar["Ftg_N"] > 0)
    & (qss_bar["Ftg_N"] < 2.5e6)
]

cmap = plt.cm.get_cmap("coolwarm")
normalize = plt.Normalize(vmin=temp["P_W"].min(), vmax=temp["P_W"].max())

sns.scatterplot(
    temp,
    x="vr_mps",
    y="Ftg_N",
    hue="P_W",
    palette=cmap,
    ax=ax,
    # marker=',',
    edgecolor=None,
    legend=False,
)

plt.ylim([0, 2.5e6])
plt.xlim([-6, 8])

sm = plt.cm.ScalarMappable(cmap=cmaps["coolwarm"], norm=normalize)
sm.set_array([])
clb = plt.colorbar(sm)
clb.ax.set_title("P_W")

ax.grid()
plt.tight_layout()
plt.savefig("../results/vrFt_SI.png")

In [ ]:
# Making the same graph in plotly is very easy.
px.scatter(temp, x="vr_mps", y="Ftg_N", color="P_W")


In [ ]:
px.scatter(temp, x="vr_mps", y="Ftg_N", color="vw_mps")


In [ ]:
temp = qss_bar[
    (qss_bar["vr_mps"] > -6)
    & (qss_bar["vr_mps"] < 8)
    & (qss_bar["Ftg_N"] > 0.20e6)
    & (qss_bar["Ftg_N"] < 0.30e6)
    # & (temp_nona['P_W'] > 0)
]

# px.scatter(temp, x="vr_mps", y="Ftg_N", color="P_W")  # Very good for intuition about what a winch controller should do.
# TODO: Make a graph showing why current strategy is not good (fixed Ft) for certain wind speed.
# px.scatter(temp, x="vr_mps", y="vw_mps", color="Ftg_N")  # Quite a straight line when using few Ft points.
px.scatter(temp, x="vr_mps", y="Ftg_N")

In [ ]:
temp = qss_bar[
    (qss_bar["vr_mps"] > -6)
    & (qss_bar["vr_mps"] < 8)
    & (qss_bar["Ftg_N"] > 0.20e6)
    & (qss_bar["Ftg_N"] < 0.30e6)
    # & (temp_nona['P_W'] > 0)
]

temp_nona = temp.dropna()

# Building the X matrix. (input data)
vr = temp_nona["vr_mps"].values
vr2 = vr**2
vw = temp_nona["vw_mps"].values
vw2 = vw**2
ones = np.ones_like(vr)
X = np.asarray([ones, vr, vw]).transpose()
# X = np.asarray([ones, vr, vw, vr**2, vw**2]).transpose()

# Outputs.
y = temp_nona["Ftg_N"].values

# Least-squares.
theta_hat = np.dot(np.dot(np.linalg.inv(np.dot(X.transpose(), X)), X.transpose()), y)
theta_hat

In [ ]:
# Doing the same with seaborn is really difficult.
fig, ax = plt.subplots(1, 1, figsize=PARTSIZE)

# df_optimal = qss_star[
#     (qss_star["Lt_m"] == Lt_m_bar)
#     & (qss_star["beta_deg"] == beta_deg_bar)
#     & (qss_star["phi_deg"] == phi_deg_bar)
#     & (qss_star["chi_deg"] == chi_deg_bar)
# ]

# sns.lineplot(
#     df_optimal,
#     x="vr_mps_star",
#     y="Ftg_N_star",
# )
# ax.legend(["Optimal"])

beta_deg_bar = 30
phi_deg_bar = 17.5
chi_deg_bar = 90
Lt_m_bar = 1000
temp = qss_bar[
    (qss_bar["vr_mps"] > -6)
    & (qss_bar["vr_mps"] < 8)
    & (qss_bar["Ftg_N"] > 0)
    & (qss_bar["Ftg_N"] < 2.5e6)
]

# cmap = plt.cm.get_cmap("coolwarm")
# normalize = plt.Normalize(vmin=temp["P_W"].min(), vmax=temp["P_W"].max())

# sns.scatterplot(
#     temp,
#     x="vr_mps",
#     y="Ftg_N",
#     hue="P_W",
#     palette=cmap,
#     ax=ax,
#     # marker=',',
#     edgecolor=None,
#     legend=False,
# )

for (vw, df) in temp.groupby("vw_mps"):
    df = df.sort_values(by="Ftg_N")
    plt.plot(df["vr_mps"], df["Ftg_N"], label=vw)

ax.legend(bbox_to_anchor=(1.1, 1.05), title="wind speed")

plt.gca().set_prop_cycle(None)
for vw in temp["vw_mps"].unique():
    vr = np.linspace(-6, 8, 100)
    Ft_hat = (
        theta_hat[0]
        + theta_hat[1] * vr
        + theta_hat[2] * vw
        # + theta_hat[3] * vr**2
        # + theta_hat[4] * vw**2
    )
    plt.plot(vr, Ft_hat, "--")

plt.ylim([0, 2.5e6])
plt.xlim([-6, 8])

# sm = plt.cm.ScalarMappable(cmap=cmaps["coolwarm"], norm=normalize)
# sm.set_array([])
# clb = plt.colorbar(sm)
# clb.ax.set_title("P_W")

plt.xlabel("vr_mps")
plt.ylabel("Ftg_N")

ax.grid()
plt.tight_layout()
plt.savefig("../results/vrFt_SI_fit.png")

## Using Plotly
It seemed super nice at first plotly express is great but with subplots I gave up. You can't use plotly expess anymore and then need to write a lot of code and the margins get screwed up (overlapping labels and figures) and it just not fitting on the width that I want (that you have to specify in pixels instead of cm) because margins around the figure are quite big.

### Loyd

In [ ]:
# Dictionary for plotly to translate variable names to labels.
# This is actually really nice.
var_to_label = {
    "beta_deg": "elevation [deg]",
    "phi_deg": "azimuth [deg]",
    "vr_mps_star": "reeling speed [m/s]",
    "f_star": "reeling factor [-]",
    "Ftg_N_star": "tether force ground [N]",
    "P_W_star": "power output [W]",
    "vw_mps": "wind speed [m/s]",
}


In [ ]:
# fFP graph for massless system for beta and phi equal to zero and a certain tether
# length.
temp = qss_massless_analytical_star[
    (qss_massless_analytical_star["beta_deg"] == 0)
    & (qss_massless_analytical_star["phi_deg"] == 0)
    & (qss_massless_analytical_star["Lt_m"] == 1000)
]

fig = make_subplots(rows=1, cols=4)

for i, y in enumerate(["f_star", "Ftg_N_star", "P_W_star"]):
    fig.add_trace(
        go.Scatter(
            x=temp["vw_mps"],
            y=temp[y],
        ),
        row=1,
        col=i + 1,
    )


fig.show()

In [ ]:
# vr_Ft graph for massless system for beta and phi equal to zero and a certain tether
# length.
temp = qss_massless_analytical_star[
    (qss_massless_analytical_star["beta_deg"] == 0)
    & (qss_massless_analytical_star["phi_deg"] == 0)
    & (qss_massless_analytical_star["Lt_m"] == 1000)
]
fig = px.line(
    temp,
    x="vr_mps_star",
    y="Ftg_N_star",
    labels=var_to_label,
)
fig.show()

In [ ]:
fig.write_image("../results/vrFt_Loyd.png")

### Elevation and azimuth

In [ ]:
# Changing beta and phi.
temp = qss_massless_analytical_star[
    qss_massless_analytical_star["Lt_m"] == 1000
].sort_values("vw_mps")


# Works when in columns but to fit on one page its totally squished.
# When in rows the ylabels overlap.
fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    # Needed to have enough margin to display ylabel. Should be automatic.
    # horizontal_spacing=0.15,
)

for i, y in enumerate(["f_star", "Ftg_N_star", "P_W_star"]):
    color_i = 0

    for beta_deg in np.unique(temp["beta_deg"]):
        for phi_deg in np.unique(temp["phi_deg"]):
            temp2 = temp[(temp["beta_deg"] == beta_deg) & (temp["phi_deg"] == phi_deg)]

            fig.add_trace(
                go.Scatter(
                    x=temp2["vw_mps"],
                    y=temp2[y],
                    line=dict(color=default_colors[color_i]),
                    # Only show legend of one subplot to get only three entries.
                    showlegend=(i == 0),
                    name=f"{beta_deg}, {phi_deg}",
                ),
                col=1,
                row=i + 1,
            )

            color_i += 1

    fig.update_yaxes(title_text=var_to_label[y], col=1, row=i + 1)

fig.update_xaxes(title_text=var_to_label["vw_mps"], col=1, row=3)
fig.update_layout(
    legend_title_text=f"{var_to_label['beta_deg']}, {var_to_label['phi_deg']}",
    yaxis={"automargin": True},
)
fig.show()


# Abuse plotly.express
# Don't have labels.
# figures = []
# for i, y in enumerate(["f_star", "Ftg_N_star", "P_W_star"]):
#     figures.append(
#         px.line(
#             temp,
#             x="vw_mps",
#             y=y,
#             color="beta_deg",
#             symbol="phi_deg",
#             labels=var_to_label,
#         )
#     )


# fig = make_subplots(rows=1, cols=len(figures))
# for i, figure in enumerate(figures):
#     for trace in range(len(figure["data"])):
#         fig.append_trace(figure["data"][trace], row=1, col=i+1)

# fig.show()

# fig_all = go.Figure(data=fig1.data + fig2.data + fig3.data)
# fig_all.show()


In [ ]:
import plotly.io as pio

pio.kaleido.default_width


In [ ]:
# I can also put the figures next to each other in latex with subcaptions.
# So just save three smaller figures.
for i, y in enumerate(["f_star", "Ftg_N_star", "P_W_star"]):
    fig = px.line(
        temp,
        x="vw_mps",
        y=y,
        color="beta_deg",
        symbol="phi_deg",
        labels=var_to_label,
    )

    if i == 2:
        fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    else:
        fig.update_layout(showlegend=False)

    fig.show()

    # With this width the figures become totally squished.
    fig.write_image(
        f"../results/fFP_betaphi_{i}.png",
        width=int(pio.kaleido.default_width / 3),
    )

In [ ]:
fig.write_image("../results/fFP_betaphi.png")


In [ ]:
# Changing beta and phi.
temp = qss_massless_analytical_star[
    qss_massless_analytical_star["Lt_m"] == 1000
].sort_values("vr_mps_star")

fig = px.line(
    temp,
    x="vr_mps_star",
    y="Ftg_N_star",
    color="beta_deg",
    symbol="phi_deg",
    labels=var_to_label,
)
fig.show()
# Very cool to see that a change in force due to a change in beta/phi is equivalent to a
# change in windspeed.


In [ ]:
fig.write_image("../results/vrFt_betaphi.png")

# OLD OLD OLD OLD

# Cleaning the data

I saw that there were some anomalies when plotting the Power or tether force over reeling factor. Clearly there were some outliers. By sorting each state by 'f' and then seeing where there is a larger than normal difference in Ftg_N, these datapoints can be taken out.

# !!!! This method only works once !!!! it doesn't work if you append to the DataFrame!
# Better to do Ftg_N.diff / f.diff() and take out values that don't fall smoothly on that line.

In [ ]:
# TODO: make this also into a python class and save as a separate csv.
# TODO: Needed? when looking at df_star I think it's actually fine already.


In [ ]:
# # Imports
# import pandas as pd
# # import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go


In [ ]:
# # Check if data looks ok.
# df = qss.df.loc[
#     (qss.df['beta_deg'] == 0)
#     & (qss.df['phi_deg'] == 0)
#     & (qss.df['chi_deg'] == 180)
#     & (qss.df['Lt_m'] == 1500)
#     & (qss.df['vw_mps'] == 10)
#     # & (qss.df['P_W'] > -1e6)
# ].sort_values(by='vr_mps')

# fig = px.line(df, x='vr_mps', y='Ftg_N', color='vw_mps', markers=True, symbol='chi_deg')
# fig.show()


In [ ]:
# df['d_Ftg_N_df'] = df['Ftg_N'].diff() / df['f'].diff()
# df['d_Ftg_N_df_diff'] = df['d_Ftg_N_df'].diff().diff().diff().diff()
# fig = px.line(df, x='f', y='d_Ftg_N_df_diff', color='vw_mps', markers=True, symbol='beta_deg')
# fig.show()

In [ ]:
# df.drop(df.loc[np.isclose(df['Ftg_N_diff'], -50000, atol=1)].index)


In [ ]:
# qss_temp.df.shape


In [ ]:
# indices_to_delete = []

# for beta_deg in qss_temp.df['beta_deg'].unique():
#     for phi_deg in qss_temp.df['phi_deg'].unique():
#         for chi_deg in qss_temp.df['chi_deg'].unique():
#             for Lt_m in qss_temp.df['Lt_m'].unique():
#                 for vw_mps in qss_temp.df['vw_mps'].unique():
#                     df = qss_temp.df[
#                         (qss_temp.df['beta_deg'] == beta_deg)
#                         & (qss_temp.df['phi_deg'] == phi_deg)
#                         & (qss_temp.df['chi_deg'] == chi_deg)
#                         & (qss_temp.df['Lt_m'] == Lt_m)
#                         & (qss_temp.df['vw_mps'] == vw_mps)
#                     ].sort_values(by='f')

#                     df['Ftg_N_diff'] = df['Ftg_N'].diff()
#                     indices_to_delete.append(df.loc[~np.isclose(df['Ftg_N_diff'], -50000, atol=1)].index)


In [ ]:
# indices_to_delete


In [ ]:
# for idx in indices_to_delete:
#     qss_temp.df.drop(idx, inplace=True)


In [ ]:
# qss.df.shape


In [ ]:
# # Check if that helped.
# df = qss_temp.df.loc[
#     # (qss_temp.df['beta_deg'] == 15)
#     (qss_temp.df['phi_deg'] == 0)
#     & (qss_temp.df['chi_deg'] == 135)
#     & (qss_temp.df['Lt_m'] == 1500)
#     # & (qss_temp.df['vw_mps'] == 15)
#     # & (qss.df['P_W'] > -1e6)
# ].sort_values(by='f')

# fig = px.line(df, x='f', y='Ftg_N', color='vw_mps', markers=True, symbol='beta_deg')
# fig.show()


In [ ]:
# Finding the best power for all states.
temp = qss.df.sort_values("P_W", ascending=False)
qss_star = temp.drop_duplicates(
    ["beta_deg", "phi_deg", "chi_deg", "Lt_m", "vw_mps"], keep="first"
)

# temp = qss_massless.df.sort_values('P_W', ascending=False)
# qss_massless_star = temp.drop_duplicates(['beta_deg', 'phi_deg', 'chi_deg', 'Lt_m', 'vw_mps'], keep='first')

# temp = qss_massless_tetherless.df.sort_values('P_W', ascending=False)
# qss_massless_tetherless_star = temp.drop_duplicates(['beta_deg', 'phi_deg', 'chi_deg', 'Lt_m', 'vw_mps'], keep='first')

# temp = qss_tetherdrag.df.sort_values('P_W', ascending=False)
# qss_tetherdrag_star = temp.drop_duplicates(['beta_deg', 'phi_deg', 'chi_deg', 'Lt_m', 'vw_mps'], keep='first')

In [ ]:
# Combine.
# TODO.


# Plotting

In [ ]:
# Very interesting. For beta=phi=0, massless and with mass are almost identical!
beta_bar = 0
phi_bar = 0

df = qss_star.loc[
    (qss_star["beta_deg"] == beta_bar)
    & (qss_star["phi_deg"] == phi_bar)
    & (qss_star["chi_deg"] == 90)
    & (qss_star["Lt_m"] == 1500)
    # & (qss_star['vw_mps'] == 15)
].sort_values(by="vr_mps")

df_massless = qss_massless_star.loc[
    (qss_massless_star["beta_deg"] == beta_bar)
    & (qss_massless_star["phi_deg"] == phi_bar)
    # & (qss_massless_star['chi_deg'] == 0)  # is always 0 for massless.
    & (qss_massless_star["Lt_m"] == 1500)
    # & (qss_massless_star['vw_mps'] == 0)
].sort_values(by="vr_mps")

df_massless_tetherless = qss_massless_tetherless_star.loc[
    (qss_massless_tetherless_star["beta_deg"] == beta_bar)
    & (qss_massless_tetherless_star["phi_deg"] == phi_bar)
    # & (qss_massless_tetherless_star['chi_deg'] == 0)  # is always 0 for massless.
    & (qss_massless_tetherless_star["Lt_m"] == 1500)
    # & (qss_massless_tetherless_star['vw_mps'] == 0)
].sort_values(by="vr_mps")

df_tetherdrag_star = qss_tetherdrag_star.loc[
    (qss_tetherdrag_star["beta_deg"] == beta_bar)
    & (qss_tetherdrag_star["phi_deg"] == phi_bar)
    # & (qss_tetherdrag_star['chi_deg'] == 0)  # is always 0 for massless.
    & (qss_tetherdrag_star["Lt_m"] == 1500)
    # & (qss_tetherdrag_star['vw_mps'] == 0)
].sort_values(by="vr_mps")


# fig1 = px.line(df, x='vr_mps', y='Ftg_N', markers=True)
# fig2 = px.line(df_massless, x='vr_mps', y='Ftg_N', markers=True)
# fig3 = px.line(df_massless_tetherless, x='vr_mps', y='Ftg_N', markers=True)

# fig_all = go.Figure(data=fig1.data + fig2.data + fig3.data)
# fig_all.show()

# Massless quick and dirty
vr = np.linspace(0, 10, 100)
f = (
    np.ones_like(vr)
    * 1
    / 3
    * np.cos(np.deg2rad(beta_bar))
    * np.cos(np.deg2rad(phi_bar))
)
Ft = (
    0.5
    * 1.225
    * (vr * 1 / f) ** 2
    * 150.45
    * 1.76
    * np.sqrt(1.0 + (1.0 / 8.4) ** 2.0)
    * (1 + 8.4**2)
    * (np.cos(np.deg2rad(beta_bar)) * np.cos(np.deg2rad(phi_bar)) - f) ** 2
)
Ft2 = (
    0.5
    * 1.225
    * (vr * 1 / f) ** 2
    * 150.45
    * 1.76
    * np.sqrt(1.0 + (1.0 / 7.17) ** 2.0)
    * (1 + 7.17**2)
    * (np.cos(np.deg2rad(beta_bar)) * np.cos(np.deg2rad(phi_bar)) - f) ** 2
)
P = Ft * vr

fig = go.Figure()

fig.add_trace(go.Scatter(x=df["vr_mps"], y=df["Ftg_N"], name="MegAWES"))
fig.add_trace(
    go.Scatter(x=df_massless["vr_mps"], y=df_massless["Ftg_N"], name="MegAWES massless")
)
fig.add_trace(
    go.Scatter(
        x=df_massless_tetherless["vr_mps"],
        y=df_massless_tetherless["Ftg_N"],
        name="MegAWES massless & tetherless",
    )
)
fig.add_trace(
    go.Scatter(
        x=df_tetherdrag_star["vr_mps"],
        y=df_tetherdrag_star["Ftg_N"],
        name="MegAWES tetherdrag",
    )
)
fig.add_trace(go.Scatter(x=vr, y=Ft, name="Analytical"))
fig.add_trace(go.Scatter(x=vr, y=Ft2, name="Analytical2"))

fig.show()

In [ ]:
fig = go.Figure()

chi_deg = 0

for vw_mps in np.unique(qss.df["vw_mps"]):
    # Check if data looks ok.
    df = qss.df.loc[
        (qss.df["beta_deg"] == 0)
        & (qss.df["phi_deg"] == 0)
        & (qss.df["chi_deg"] == chi_deg)
        & (qss.df["Lt_m"] == 1500)
        & (qss.df["vw_mps"] == vw_mps)
        # & (qss.df['P_W'] > -1e6)
    ].sort_values(by="vr_mps")

    fig.add_trace(go.Scatter(x=df["vr_mps"], y=df["P_W"], name=f"{vw_mps=}"))

df = qss_star.loc[
    (qss_star["beta_deg"] == 0)
    & (qss_star["phi_deg"] == 0)
    & (qss_star["chi_deg"] == chi_deg)
    & (qss_star["Lt_m"] == 1500)
    # & (qss_star['vw_mps'] == 15)
].sort_values(by="vr_mps")
fig.add_trace(go.Scatter(x=df["vr_mps"], y=df["P_W"], name="optimum"))
fig.show()